In [1]:
import re
import pandas as pd
from features_prep import *

In [51]:
df = pd.read_csv('./MP_datasets/Kickstarter20_21_short3.csv', index_col=0)

In [52]:
df = df.reset_index(drop=True)

In [37]:
# df.info()

In [53]:
df['category'].convert_dtypes(convert_string=True)
df['subcategory'] = df['category'].apply(lambda x: subcatego(x))
df['category'] = df['category'].apply(lambda x: catego(x))

In [54]:
df = wrangle2(df)

In [55]:
df.columns

Index(['category', 'country', 'created_at', 'deadline', 'id', 'launched_at',
       'success', 'subcategory', 'goal_usd', 'launch_day',
       'campaign_duration_in_days', 'name_blurb'],
      dtype='object')

In [56]:
df.head(4)

,category,country,created_at,deadline,id,launched_at,success,subcategory,goal_usd,launch_day,campaign_duration_in_days,name_blurb
1,publishing,US,2017-02-07 21:29:49,2017-04-17 03:55:15,1920284335,2017-03-18 03:55:15,1,art books,3700.00,5,30,"(Hand Crafted: An Illustrated Cocktail Book, A..."
2,film,GB,2019-11-25 20:46:29,2020-01-04 23:59:00,792653686,2019-12-06 20:24:11,1,film,2622.77,4,29,"(The Road Home, Exploring the disastrous socia..."
3,technology,HK,2019-10-21 03:03:23,2020-01-08 15:00:31,68439695,2019-12-09 15:00:31,1,hardware,2043.41,0,30,"(Gkempt：56TB HDD Cloud Storage, No Subscriptio..."
4,film,US,2015-05-08 18:49:11,2015-07-01 00:00:00,1935147814,2015-06-01 06:24:04,1,film,15000.00,0,30,"(Souls of the Vermilion Sea, A documentary fil..."


In [57]:
df_narrow = df[df.columns[[4, 6, 8, 10, 0, 7, 1, 9, 11]]] 

In [58]:
df_narrow.head()

,id,success,goal_usd,campaign_duration_in_days,category,subcategory,country,launch_day,name_blurb
1,1920284335,1,3700.00,30,publishing,art books,US,5,"(Hand Crafted: An Illustrated Cocktail Book, A..."
2,792653686,1,2622.77,29,film,film,GB,4,"(The Road Home, Exploring the disastrous socia..."
3,68439695,1,2043.41,30,technology,hardware,HK,0,"(Gkempt：56TB HDD Cloud Storage, No Subscriptio..."
4,1935147814,1,15000.00,30,film,film,US,0,"(Souls of the Vermilion Sea, A documentary fil..."
5,1785881232,1,3878.16,30,technology,technology,GB,1,(Pop Lumen! The magnetic quick release keychai...


In [46]:
((7000-780)/2)/3600

0.8638888888888889

In [59]:
blurb_stats = df_narrow['name_blurb'].apply(lambda x: blurb_quant(x))
# 7000 s for 280000 rows
#  780 s for  27000 rows
# 2740 s for  13000 rows
# 2870 s for  13000 rows

In [13]:
df_narrow.subcategory.unique  

# MP: spacy-vectorize the subcategories (adjusted: rock --> rock music, diy --> d.i.y. crafts, etc.), 
# feed neural network;
# dimensionality reduction is ok, if it helps, because front-end will explain the feature importance without numbers

<bound method Series.unique of 0         conceptual art
1                   rock
2         conceptual art
3                    diy
4         graphic novels
              ...       
27497              zines
27498        3d printing
27499    diy electronics
27500               food
27501            gadgets
Name: subcategory, Length: 26951, dtype: object>

In [60]:
df_wide3 = df_narrow.join(blurb_stats)


In [61]:
df_wide3 = df_wide3.drop('name_blurb', axis=1)

In [62]:
# df_wide.to_csv('df_wide.csv', index_label=False)  # not ok
# df_wide.to_csv('df_wide.csv', index_label=None)   # ok
df_wide3.to_csv('df_wide3.csv', index=False)  # yes


In [26]:
df_wide.tail()

,id,success,goal_usd,campaign_duration_in_days,category,subcategory,country,launch_day,name_chars,name_words,...,noun,shout,capwords,norm,subj,pol,pos,neg,neu,comp
27497,888190580,0,2424.97,30,publishing,zines,NZ,6,58.0,11.0,...,9.0,0.0,6.0,3.269701,0.950000,0.812500,0.301,0.000,0.699,0.8478
27498,495116649,0,23363.01,35,technology,3d printing,DE,3,55.0,10.0,...,14.0,0.0,2.0,3.056597,0.500000,0.500000,0.139,0.000,0.861,0.4404
27499,831728453,0,4440.73,30,technology,diy electronics,GB,4,60.0,10.0,...,7.0,1.0,8.0,2.862655,0.000000,0.000000,0.000,0.000,1.000,0.0000
27500,935071903,1,8500.00,30,food,food,US,0,20.0,3.0,...,5.0,0.0,4.0,3.451262,0.497727,0.163636,0.216,0.000,0.784,0.7003
27501,1914160519,0,225000.00,45,technology,gadgets,US,1,59.0,10.0,...,4.0,4.0,12.0,3.175325,0.500000,0.500000,0.118,0.094,0.787,-0.1027


In [65]:
df_wide.shape

(26951, 27)

In [64]:
df_wide2.shape

(94255, 27)

In [66]:
df_wide3.shape

(100029, 27)

In [92]:
df_tue = pd.concat([df_wide, df_wide2, df_wide3], axis=0)

In [93]:
df_tue.shape

(221235, 27)

In [125]:
df_shuffled = df_tue.sample(frac=1, axis=0, random_state=33)

train = int(len(df_shuffled)*0.7)
train_df = df_shuffled[:train]
holdout_df = df_shuffled[train:]

third = round(len(holdout_df)/3)
test_df = holdout_df[:third]
dev_df = holdout_df[third:2*third]
val_df = holdout_df[2*third:]

In [149]:
train_df.to_csv('train_df.csv', index=False)
val_df.to_csv('val_df.csv', index=False)
dev_df.to_csv('dev_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)